In [1]:
import pandas as pd
import numpy as np
import re
import string

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import spacy

In [2]:
route = '../../../data/'

In [3]:
df = pd.read_csv(route + 'datamarket_productos_de_supermercados.csv')
df.head()

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id
0,https://tienda.mercadona.es/product/46497/prot...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Protector solar Seesee FPS 30,Spray,5.0,2.50,100ml,2021-03-06 00:00:00,0d4d08c08fc91dda571901f5db354c19
1,https://tienda.mercadona.es/product/41178/lech...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Leche protección solar Sun Med FPS 30+,Bote,4.0,1.33,100ml,2021-03-06 00:00:00,835a957da758d6143667a9e7baccb88e
2,https://tienda.mercadona.es/product/76915/crem...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Crema facial con color BBsun FPS 50 Delial ton...,Bote,8.9,17.80,100ml,2021-03-06 00:00:00,a28ae5555e8fcc9ba73ec9f98bc2a5fe
3,https://tienda.mercadona.es/product/72116/gel-...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Gel hidratante solar rostro y escote Delial FP...,Bote,11.0,22.00,100ml,2021-03-06 00:00:00,89a8e9cb51a3b1ed455cdd8ea5b86101
4,https://tienda.mercadona.es/product/78799/prot...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Protector solar niños Delial FPS 50+,Spray,10.6,5.30,100ml,2021-03-06 00:00:00,8f724d7cd7c428b7fe1a96032ff90db2


In [4]:
df.info()

# Analizar si 'insert_date' hay que cambiarlo a formato fecha

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8893237 entries, 0 to 8893236
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   url              object 
 1   supermarket      object 
 2   category         object 
 3   name             object 
 4   description      object 
 5   price            float64
 6   reference_price  float64
 7   reference_unit   object 
 8   insert_date      object 
 9   product_id       object 
dtypes: float64(2), object(8)
memory usage: 678.5+ MB


# Modificando formato de columna

In [5]:
# Creamos nueva columna 'Date' para que adquiera el formato Datetime

df["Date"] = pd.to_datetime(df["insert_date"], format="%Y/%m/%d")
df.head()

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Date
0,https://tienda.mercadona.es/product/46497/prot...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Protector solar Seesee FPS 30,Spray,5.0,2.50,100ml,2021-03-06 00:00:00,0d4d08c08fc91dda571901f5db354c19,2021-03-06
1,https://tienda.mercadona.es/product/41178/lech...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Leche protección solar Sun Med FPS 30+,Bote,4.0,1.33,100ml,2021-03-06 00:00:00,835a957da758d6143667a9e7baccb88e,2021-03-06
2,https://tienda.mercadona.es/product/76915/crem...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Crema facial con color BBsun FPS 50 Delial ton...,Bote,8.9,17.80,100ml,2021-03-06 00:00:00,a28ae5555e8fcc9ba73ec9f98bc2a5fe,2021-03-06
3,https://tienda.mercadona.es/product/72116/gel-...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Gel hidratante solar rostro y escote Delial FP...,Bote,11.0,22.00,100ml,2021-03-06 00:00:00,89a8e9cb51a3b1ed455cdd8ea5b86101,2021-03-06
4,https://tienda.mercadona.es/product/78799/prot...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Protector solar niños Delial FPS 50+,Spray,10.6,5.30,100ml,2021-03-06 00:00:00,8f724d7cd7c428b7fe1a96032ff90db2,2021-03-06


In [6]:
df.drop('insert_date', axis=1, inplace=True)

In [7]:
df.sort_values(by='Date')
df

,url,supermarket,category,name,description,price,reference_price,reference_unit,product_id,Date
0,https://tienda.mercadona.es/product/46497/prot...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Protector solar Seesee FPS 30,Spray,5.00,2.50,100ml,0d4d08c08fc91dda571901f5db354c19,2021-03-06
1,https://tienda.mercadona.es/product/41178/lech...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Leche protección solar Sun Med FPS 30+,Bote,4.00,1.33,100ml,835a957da758d6143667a9e7baccb88e,2021-03-06
2,https://tienda.mercadona.es/product/76915/crem...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Crema facial con color BBsun FPS 50 Delial ton...,Bote,8.90,17.80,100ml,a28ae5555e8fcc9ba73ec9f98bc2a5fe,2021-03-06
3,https://tienda.mercadona.es/product/72116/gel-...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Gel hidratante solar rostro y escote Delial FP...,Bote,11.00,22.00,100ml,89a8e9cb51a3b1ed455cdd8ea5b86101,2021-03-06
4,https://tienda.mercadona.es/product/78799/prot...,mercadona-es,cuidado_facial_y_corporal_protector_solar_y_af...,Protector solar niños Delial FPS 50+,Spray,10.60,5.30,100ml,8f724d7cd7c428b7fe1a96032ff90db2,2021-03-06
...,...,...,...,...,...,...,...,...,...,...
8893232,https://www.carrefour.es/supermercado/empapado...,carrefour-es,mascotas_perros_confort,Empapadores para Perros 60x60cm,NaN,2.95,2.95,ud,399b565a8079db4fab86c0dab48dcdb9,2022-11-04
8893233,https://www.carrefour.es/supermercado/comedero...,carrefour-es,mascotas_perros_confort,Comedero Antideslizante para Perro Carrefour 5...,NaN,2.49,2.49,ud,f6dfa4a9185bd527d4dcabae88980d3b,2022-11-04
8893234,https://www.carrefour.es/supermercado/snacks-d...,carrefour-es,mascotas_perros_higiene,Snacks dental para perros de 5-15 kg Pedigree ...,NaN,1.79,26.32,kg,9f776b1a31967e6ab297f750658db6e4,2022-11-04
8893235,https://www.carrefour.es/supermercado/champu-3...,carrefour-es,mascotas_perros_higiene,Champú 3en1 Dixie 500 ml,NaN,3.99,3.99,ud,7a1a8b3311aa824484b06c6d310544f0,2022-11-04


# Buscando df de productos únicos

In [ ]:
df.nunique()

# llamativo que el número de product_id sea diferente a 'name'. En este caso, al ser menor y no haber nulos se podría 
# interpretar que hay varios productos con el mismo ID, ya sea xq el nombre del producto las compañías lo modifican por
# temas comerciales pero mantienen el ID.

In [ ]:
# Analizando si el id de un producto se pudiera encontrar en más de 1 supermercado.

print(df[df['product_id']=='34d9c8bf7db90e988219cef30b3cf02a'].name.unique())
print(df[df['product_id']=='34d9c8bf7db90e988219cef30b3cf02a'].name.nunique())
print(df[df['product_id']=='34d9c8bf7db90e988219cef30b3cf02a'].supermarket.unique())

In [ ]:
df_mercadona = df[df['supermarket']=='mercadona-es']
df_dia = df[df['supermarket']=='dia-es']
df_carrefour = df[df['supermarket']=='carrefour-es']

In [ ]:
df_mercadona.info()

In [ ]:
df_mercadona.drop_duplicates(subset='product_id', keep="last", inplace=True)

In [ ]:
df_mercadona['product_id'].nunique()

In [ ]:
df_dia.drop_duplicates(subset='product_id', keep="last", inplace=True)
df_carrefour.drop_duplicates(subset='product_id', keep="last", inplace=True)

In [ ]:
# Entre Carrefour y Dia no coincide ninguno

df_carrefour.merge(df_dia, on='product_id')

In [ ]:
# Entre Carrefour y Mercadona no coincide ninguno

df_carrefour.merge(df_mercadona, on='product_id')

In [ ]:
# Entre Dia y Mercadona no coincide ninguno

df_dia.merge(df_mercadona, on='product_id')

### Eliminamos duplicados del df original

In [8]:
# Ahora que tenemos la certeza lo que hacemos es quitar todos los id duplicados
df.drop_duplicates(subset='product_id', keep="last", inplace=True)

In [9]:
df

,url,supermarket,category,name,description,price,reference_price,reference_unit,product_id,Date
487,https://tienda.mercadona.es/product/46715/dent...,mercadona-es,cuidado_facial_y_corporal_higiene_bucal,Dentífrico gel infantil + 2 años Deliplus Shim...,Tubo,1.00,1.33,100ml,39c36d19348347972ee48ddccbc4d5ea,2021-03-06
2381,https://www.dia.es/compra-online/productos/cha...,dia-es,charcuteria_y_quesos_quesos_tablas,TIGER queso fondue classic caja 400 gr,NaN,3.85,9.63,kg,8ae39e814a97b892769f85ad15990bbf,2021-03-06
2530,https://www.dia.es/compra-online/productos/cha...,dia-es,charcuteria_y_quesos_curado_fuet_y_longaniza,ARGAL fuet calidad extra envase 120 gr,NaN,1.25,11.36,kg,488442529c0030bf98bd77225e357138,2021-03-06
2893,https://www.dia.es/compra-online/productos/fre...,dia-es,frescos_carne_y_aves_hamburguesas_y_picadas,ROLER hamburguesas de lomo de vaca bandeja 2 u...,NaN,2.95,12.29,kg,d71662b3ba875e702a5d5d3646859e30,2021-03-06
4647,https://www.dia.es/compra-online/productos/dro...,dia-es,drogueria_y_limpieza_cuidado_ropa__detergente_...,COLON detergente máquina líquido flor botella ...,NaN,5.99,0.19,lavado,cd8a878448ae2292c0513aad07abcd37,2021-03-06
...,...,...,...,...,...,...,...,...,...,...
8893232,https://www.carrefour.es/supermercado/empapado...,carrefour-es,mascotas_perros_confort,Empapadores para Perros 60x60cm,NaN,2.95,2.95,ud,399b565a8079db4fab86c0dab48dcdb9,2022-11-04
8893233,https://www.carrefour.es/supermercado/comedero...,carrefour-es,mascotas_perros_confort,Comedero Antideslizante para Perro Carrefour 5...,NaN,2.49,2.49,ud,f6dfa4a9185bd527d4dcabae88980d3b,2022-11-04
8893234,https://www.carrefour.es/supermercado/snacks-d...,carrefour-es,mascotas_perros_higiene,Snacks dental para perros de 5-15 kg Pedigree ...,NaN,1.79,26.32,kg,9f776b1a31967e6ab297f750658db6e4,2022-11-04
8893235,https://www.carrefour.es/supermercado/champu-3...,carrefour-es,mascotas_perros_higiene,Champú 3en1 Dixie 500 ml,NaN,3.99,3.99,ud,7a1a8b3311aa824484b06c6d310544f0,2022-11-04


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34836 entries, 487 to 8893236
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   url              34836 non-null  object        
 1   supermarket      34836 non-null  object        
 2   category         34836 non-null  object        
 3   name             34823 non-null  object        
 4   description      7751 non-null   object        
 5   price            34836 non-null  float64       
 6   reference_price  34836 non-null  float64       
 7   reference_unit   34813 non-null  object        
 8   product_id       34836 non-null  object        
 9   Date             34836 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 2.9+ MB


In [11]:
df.nunique()

url                34836
supermarket            3
category            1039
name               33247
description           41
price               1672
reference_price     5047
reference_unit         9
product_id         34836
Date                 609
dtype: int64

# Gestión de Nulos

In [12]:
df.isnull().sum()

url                    0
supermarket            0
category               0
name                  13
description        27085
price                  0
reference_price        0
reference_unit        23
product_id             0
Date                   0
dtype: int64

In [13]:
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

name                 13
description       27085
reference_unit       23
dtype: int64

In [14]:
null_cols = null_cols[null_cols > 0] / len(df) * 100
null_cols

# El más representativo es la descripción que faltan más del 63% pero no es un campo relevante para nuestro análisis
# Por otro lado están aquellos productos que sin tener nombre o precio, sí que nos podrían sobrar aunque representan un %
# muy bajo de todo el df

name               0.037318
description       77.750029
reference_unit     0.066024
dtype: float64

### Analizando 'Description' 

In [15]:
print('Tipo de valores únicos:', df['description'].unique())
print('Nº de valores únicos:', df['description'].nunique() )

Tipo de valores únicos: ['Tubo' nan 'Caja' 'Paquete' 'Bote' 'Botella' 'Pieza' 'Bandeja' 'Pack-6'
 'Lata' 'Brick' 'Pack-12' 'Tarro' 'Spray' 'Garrafa' 'Frasco' 'Pack-2'
 'Botellín' 'Tarrina' 'Malla' 'Pack-4' 'Tableta' 'Vaso' 'Pack-3' 'Pack-8'
 'Pack-10' 'Granel' 'Bol' 'Sobre' 'Benjamín' 'Pack-5' 'Tarrito' 'Saco'
 '1/2 Pieza' 'Cubo' 'Bolsa' 'Manojo' 'Pack-9' 'Barril' '1/4 Pieza'
 'Pastilla' 'Pack-16']
Nº de valores únicos: 41


In [16]:
# Podríamos concluir que aunque el número de valores únicos no es excesivamente alto (41), sí es cierto que siguen faltando
# más del 63% de descripciones en el dataset y condicionar posibles búsquedas a este campo tal vez ahora no renta.

df.drop('description',axis=1, inplace=True)
df.head()

,url,supermarket,category,name,price,reference_price,reference_unit,product_id,Date
487,https://tienda.mercadona.es/product/46715/dent...,mercadona-es,cuidado_facial_y_corporal_higiene_bucal,Dentífrico gel infantil + 2 años Deliplus Shim...,1.00,1.33,100ml,39c36d19348347972ee48ddccbc4d5ea,2021-03-06
2381,https://www.dia.es/compra-online/productos/cha...,dia-es,charcuteria_y_quesos_quesos_tablas,TIGER queso fondue classic caja 400 gr,3.85,9.63,kg,8ae39e814a97b892769f85ad15990bbf,2021-03-06
2530,https://www.dia.es/compra-online/productos/cha...,dia-es,charcuteria_y_quesos_curado_fuet_y_longaniza,ARGAL fuet calidad extra envase 120 gr,1.25,11.36,kg,488442529c0030bf98bd77225e357138,2021-03-06
2893,https://www.dia.es/compra-online/productos/fre...,dia-es,frescos_carne_y_aves_hamburguesas_y_picadas,ROLER hamburguesas de lomo de vaca bandeja 2 u...,2.95,12.29,kg,d71662b3ba875e702a5d5d3646859e30,2021-03-06
4647,https://www.dia.es/compra-online/productos/dro...,dia-es,drogueria_y_limpieza_cuidado_ropa__detergente_...,COLON detergente máquina líquido flor botella ...,5.99,0.19,lavado,cd8a878448ae2292c0513aad07abcd37,2021-03-06


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34836 entries, 487 to 8893236
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   url              34836 non-null  object        
 1   supermarket      34836 non-null  object        
 2   category         34836 non-null  object        
 3   name             34823 non-null  object        
 4   price            34836 non-null  float64       
 5   reference_price  34836 non-null  float64       
 6   reference_unit   34813 non-null  object        
 7   product_id       34836 non-null  object        
 8   Date             34836 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 2.7+ MB


### Analizando Nulos de precios

In [18]:
df = df[df['name'].notna()]

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34823 entries, 487 to 8893236
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   url              34823 non-null  object        
 1   supermarket      34823 non-null  object        
 2   category         34823 non-null  object        
 3   name             34823 non-null  object        
 4   price            34823 non-null  float64       
 5   reference_price  34823 non-null  float64       
 6   reference_unit   34800 non-null  object        
 7   product_id       34823 non-null  object        
 8   Date             34823 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 2.7+ MB


In [20]:
df.isnull().sum()

url                 0
supermarket         0
category            0
name                0
price               0
reference_price     0
reference_unit     23
product_id          0
Date                0
dtype: int64

# Analizar qué categorías me interesan o hay que quitar

In [21]:
sorted(df['category'].unique().tolist())
#print(df['category'].nunique())

['aceite_especias_y_salsas_aceite_vinagre_y_sal',
 'aceite_especias_y_salsas_especias',
 'aceite_especias_y_salsas_mayonesa_ketchup_y_mostaza',
 'aceite_especias_y_salsas_otras_salsas',
 'agua_y_refrescos_agua',
 'agua_y_refrescos_isotonico_y_energetico',
 'agua_y_refrescos_refresco_de_cola',
 'agua_y_refrescos_refresco_de_naranja_y_de_limon',
 'agua_y_refrescos_refresco_de_te_y_sin_gas',
 'agua_y_refrescos_tonica_y_bitter',
 'al_dia_carne_cerdo',
 'al_dia_carne_mixto',
 'al_dia_carne_pavo',
 'al_dia_carne_pollo',
 'al_dia_carne_vacuno',
 'al_dia_charcuteria_y_quesos_cocidos',
 'al_dia_charcuteria_y_quesos_curados',
 'al_dia_charcuteria_y_quesos_quesos',
 'al_dia_frutas_citricos',
 'al_dia_frutas_frutas_de_temporada',
 'al_dia_frutas_frutas_tropicales',
 'al_dia_frutas_manzanas',
 'al_dia_frutas_peras',
 'al_dia_frutas_platanos',
 'al_dia_pescado_y_marisco_ahumados_y_salazones',
 'al_dia_pescado_y_marisco_marisco',
 'al_dia_pescado_y_marisco_pescado_fresco',
 'al_dia_pescado_y_marisco_

In [22]:
# Empezamos asegurándonos lo q contiene el str 'higie'

df_clean = df[df["category"].str.contains("higie|hogar|limpieza|farmacia|cuidado|bebe|mascot|maquillaj|solidari|perfume") == False]
sorted(df_clean['category'].unique().tolist())
print(df_clean['category'].nunique())
#df_clean['category'].value_counts()

644


In [23]:
df_clean
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19554 entries, 2381 to 8892975
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   url              19554 non-null  object        
 1   supermarket      19554 non-null  object        
 2   category         19554 non-null  object        
 3   name             19554 non-null  object        
 4   price            19554 non-null  float64       
 5   reference_price  19554 non-null  float64       
 6   reference_unit   19551 non-null  object        
 7   product_id       19554 non-null  object        
 8   Date             19554 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 1.5+ MB


In [24]:
sorted(df_clean['category'].unique().tolist())

['aceite_especias_y_salsas_aceite_vinagre_y_sal',
 'aceite_especias_y_salsas_especias',
 'aceite_especias_y_salsas_mayonesa_ketchup_y_mostaza',
 'aceite_especias_y_salsas_otras_salsas',
 'agua_y_refrescos_agua',
 'agua_y_refrescos_isotonico_y_energetico',
 'agua_y_refrescos_refresco_de_cola',
 'agua_y_refrescos_refresco_de_naranja_y_de_limon',
 'agua_y_refrescos_refresco_de_te_y_sin_gas',
 'agua_y_refrescos_tonica_y_bitter',
 'al_dia_carne_cerdo',
 'al_dia_carne_mixto',
 'al_dia_carne_pavo',
 'al_dia_carne_pollo',
 'al_dia_carne_vacuno',
 'al_dia_charcuteria_y_quesos_cocidos',
 'al_dia_charcuteria_y_quesos_curados',
 'al_dia_charcuteria_y_quesos_quesos',
 'al_dia_frutas_citricos',
 'al_dia_frutas_frutas_de_temporada',
 'al_dia_frutas_frutas_tropicales',
 'al_dia_frutas_manzanas',
 'al_dia_frutas_peras',
 'al_dia_frutas_platanos',
 'al_dia_pescado_y_marisco_ahumados_y_salazones',
 'al_dia_pescado_y_marisco_marisco',
 'al_dia_pescado_y_marisco_pescado_fresco',
 'al_dia_pescado_y_marisco_

In [25]:
df_clean.to_csv(route + 'datamarket_clean_unique.csv')